In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

import numpy as np
import cv2

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim

from data_utils import load_info
from skimage import io
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchvision import  models
from torch.optim import lr_scheduler

In [2]:
class FlowDataset(Dataset):
    def __init__(self,list_dir,img_dir,transform=None): #data_len = array
        self.list_dir = list_dir
        self.img_dir = img_dir
        self.info_list = load_info(list_dir)
        self.transform = transform

    def __len__(self):
        return len(self.info_list)

    def __getitem__(self, idx):
        info = self.info_list[idx]
        img_name = info[0]
        image = io.imread(self.img_dir+img_name)
        np_img = np.array(image)
        np_img = cv2.resize(np_img,(224,224),interpolation=cv2.INTER_AREA)
        if self.transform:
            np_img = self.transform(np_img)

        #np.load(np_img)
        #print(np_img.shape)
        #x = torch.FloatTensor(np_img.transpose(2,0,1))
        x = np_img
        val = 0
        if int(info[1]) > 2:
            val = 2
        else :
            val = 1
        y = torch.LongTensor(np.array(float(val-1)))

        return [x, y]

In [3]:
writer = SummaryWriter('runs/0903_test')

In [4]:
trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [5]:
train_batch_size= 6
train_dataset = FlowDataset("train.txt","./flow_images/",transform=trans)
dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

val_dataset = FlowDataset("val.txt", "./flow_images/", transform=trans)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

In [8]:
# torch device setting
GPU_NUM = 0  # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device)  # change allocation of current GPU
# print('Current cuda device ', torch.cuda.current_device())  # check

In [10]:
# load res18 architecture, custom last layer
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_fceatures
model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft = model_ft.to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(),lr=0.001,momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

In [ ]:
num_epochs = 1
for epoch in range(0,num_epochs):
    for i, samples in enumerate(dataloader):
        inputs, labels = samples
        inputs = Variable(inputs).to(device)
        labels = Variable(labels).to(device)
        
        model_ft.train()
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        optimizer_ft.zero_grad()
        loss.backward()
        optimizer_ft.step()
        
        print('Epoch {:4d}/{} Batch {}/{} Loss: {:.6f}'.format(
                 epoch, num_epochs, i + 1, len(dataloader),
                 loss.item()
             ))

        ## validation
        if (i + 1) % 1000 == 0 and i != 0:  # every 100 mini-batches
            with torch.no_grad():
                val_loss = 0.0
                    for j, val in enumerate(val_dataloader):
                        val_x, val_label = val







        

Epoch    0/1 Batch 1/1363 Loss: 0.112374
Epoch    0/1 Batch 2/1363 Loss: 1.906493
Epoch    0/1 Batch 3/1363 Loss: 0.133784
Epoch    0/1 Batch 4/1363 Loss: 0.328904
Epoch    0/1 Batch 5/1363 Loss: 1.055768
Epoch    0/1 Batch 6/1363 Loss: 0.057145
Epoch    0/1 Batch 7/1363 Loss: 0.024199
Epoch    0/1 Batch 8/1363 Loss: 0.016602
Epoch    0/1 Batch 9/1363 Loss: 0.362507
Epoch    0/1 Batch 10/1363 Loss: 0.013804
Epoch    0/1 Batch 11/1363 Loss: 0.028005
Epoch    0/1 Batch 12/1363 Loss: 0.010584
Epoch    0/1 Batch 13/1363 Loss: 0.096651
Epoch    0/1 Batch 14/1363 Loss: 0.082570
Epoch    0/1 Batch 15/1363 Loss: 0.023216
Epoch    0/1 Batch 16/1363 Loss: 0.022907
Epoch    0/1 Batch 17/1363 Loss: 0.299044
Epoch    0/1 Batch 18/1363 Loss: 0.008176
Epoch    0/1 Batch 19/1363 Loss: 0.069371
Epoch    0/1 Batch 20/1363 Loss: 0.013386
Epoch    0/1 Batch 21/1363 Loss: 0.118341
Epoch    0/1 Batch 22/1363 Loss: 1.483982
Epoch    0/1 Batch 23/1363 Loss: 0.049460
Epoch    0/1 Batch 24/1363 Loss: 0.537549
E